In [ ]:
!nvidia-smi

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U hf_transfer
!pip install -q -U "huggingface_hub[cli]"

In [3]:
import torch
import os
import transformers
import getpass
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
from IPython.display import display, Markdown, Latex


os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

In [ ]:
!huggingface-cli login # for gated models

In [5]:
torch.set_default_device('cuda')

In [ ]:
model_id = "google/gemma-3-4b-it"

quantization_config = BitsAndBytesConfig(load_in_4bit=True)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    low_cpu_mem_usage=True,
    torch_dtype="auto",
    device_map="auto"
    )

tokenizer = AutoTokenizer.from_pretrained(model_id)
model.device

In [ ]:
chat = [
    { "role": "user", "content": "Hi there" },
]

prompt = tokenizer.apply_chat_template(
    chat,
    tokenize=False,
    add_generation_prompt=True
    )
print(prompt)

In [8]:
inputs = tokenizer.encode(
    prompt,
    add_special_tokens=True,
    return_tensors="pt"
    )
outputs = model.generate(
    input_ids=inputs.to(model.device),
    max_new_tokens=100
    )
text = tokenizer.decode(
    outputs[0],
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True
    )

In [ ]:
display(Markdown(text))